**TODO**: plot data


## Related definitions for this topic

### Moneyness of an option

In the money: with the current spot price of the underlying the option will pay

Out of the money: $\,S_0<K\,$ for call and $\,S_0>K\,$ for put.

At the money (ATM): $\,S_0=K\,$.

### Over the counter (OTC)

Contracts are usually made
* either `at exchanges` through marketmakers, with fixed conditions and reduced risk,
* or directly between two sides, that is, `OTC` (over-the-counter), with tailored conditions and higher risk.

### Open interest

For a standardized contract type, this is the number of currently active contracts of this type.

## Observe and Download the data

1. Go to Yahoo Finance > Microsoft > Options at
<br/>https://finance.yahoo.com/quote/MSFT/options


2. Read (with your eyes) the spot price from the html page and write it into PAR['spot'] at the top of the code below.


3. Observe that each row displays information about one standardized contract type.


4. Above the top left of the "Calls" table there is a date dropdown.
<br/>Select the date that is closest to your current date.


5. Download the webpage as a single html to the `data` subfolder of your current folder.
<br/>
The name of the downloaded file is "Microsoft Corporation (MSFT) Options Chain - Yahoo Finance.html".


## Implied volatility and Open interest

Process the downloaded html file with the code below.


In [9]:
from os.path import join
import re
from collections import OrderedDict
from glob import glob
from matplotlib import pyplot as plt

PAR = {
    'spot': 320.10,  # write the spot price here
    'data_dir': 'data',
    'stock_ticker': 'MSFT',
    'data_file_name': 'Microsoft Corporation (MSFT) Options Chain - Yahoo Finance.html',
    'figure_size': (15, 8),
    'option_types': ['call', 'put'],
    'color_list': ['red', 'blue'],
    'fontsize': 14,
    'label_pad': 15,
    'ticks_fontsize': 13,
    'subplots_width_space': 0.025,
    'suplots_height_space': 0.05,
    'spot_color': 'gray',
    'spot_line_width': 1,
    'spot_dash_length': (5, 4)    
}


def read_options_data(par=None):
    """
    Read and process options data from a html file saved from finance.yahoo.com
    """

    options_data = {option_type: [] for option_type in par['option_types']}
    infile = join(par['data_dir'], par['data_file_name'])

    with open(infile, encoding='utf8') as f:
        html = f.read()
        options_match = re.match('^.*?<table class=\"calls.+?<tbody.*?>(?P<call>.*?)<\/tbody>'
                                 '.*?<table class=\"puts.+?<tbody.*?>(?P<put>.*?)<\/tbody>', 
                                 html, re.DOTALL)  # matches saved by names: call, put
        if options_match:
            html_tables = {c: options_match.group(c) for c in par['option_types']}
            for option_type, option_table_body in html_tables.items():
                for table_line_match in re.finditer(r'<td class=\"data-col2.*?><a.*?>(.+?)<\/a>.*?'
                                                    r'<td class=\"data-col9.*?>(.+?)<\/td>.*?'
                                                    r'<td class=\"data-col10.*?>(.+?)\%<\/td>',
                                                    option_table_body):
                    strike, open_interest, implied_volatility = (
                        [x.replace(',', '') for x in table_line_match.group(1, 2, 3)])
                    options_data[option_type].append({
                        'strike': float(strike),
                        'open interest': None if open_interest == '-' else int(open_interest),
                        'implied volatility': round(float(implied_volatility) * 0.01, 4)
                    })
        else:
            raise ValueError('Cannot find call and put tables in {}'.format(infile))

    return options_data

            
def plot_options_implied_volatility(par=None, options_data=None):
    """
    Plot an analysis of the implied volatility
    """
    
    # --- settings ---
    fig, axes = plt.subplots(ncols=2, nrows=2, figsize=FIGURE_SIZE)    
    [plt.rc(t, labelsize=TICKS_FONTSIZE) for t in ['xtick', 'ytick']]
    implied_vol_plot_max = {}
    open_interest_plot_limits = {}
    strike_plot_limits = []
    
    # --- compiling data and plotting ---
    for column_index, option_type in enumerate(OPTION_TYPES):

        # --- implied volatility ---
        ax = axes[0][column_index]
        strikes = []
        impl_vols = []
        for dat in options_data[option_type]:
            if dat['implied volatility'] is not None and dat['implied volatility'] > 0.0:
                strikes.append(dat['strike'])
                impl_vols.append(dat['implied volatility'])
        ax.axvline(spot, color=SPOT_COLOR, linewidth=SPOT_LINE_WIDTH, linestyle='--', dashes=SPOT_DASH_LENGTH)
        ax.plot(strikes, impl_vols, 'r-', label='Implied volatility', 
                color=COLOR_LIST[column_index], marker='^', linewidth=0, markersize=7)
        
        implied_vol_plot_max[option_type] = ax.get_ylim()[1]  
        strike_plot_limits.append(ax.get_xlim())
        ax.set_title('{} options on {} (data description: please see text)'
                     .format(option_type.capitalize(), STOCK_TICKER), fontsize=MY_FONTSIZE)
        if column_index == 0:  # show the y axis label only on the left subplot
            ax.set_ylabel('Implied volatility', fontsize=MY_FONTSIZE, labelpad=LABEL_PAD)
        else:  # hide y ticks if we are not at the left subplot
            ax.tick_params(axis='y', which='both', left=True, right=False, labelleft=False)
        # hide x ticks in the first row
        ax.tick_params(axis='x', which='both', top=False, bottom=True, labelbottom=False)
            
        # --- open interest ---
        ax = axes[1][column_index]
        strikes = []
        open_interests = []
        for dat in options_data[option_type]:
            if dat['open interest'] is not None and dat['open interest'] > 0:
                strikes.append(dat['strike'])
                open_interests.append(dat['open interest'])
        ax.set_yscale('log')
        ax.axvline(spot, color=SPOT_COLOR, linewidth=SPOT_LINE_WIDTH, linestyle='--', dashes=SPOT_DASH_LENGTH)
        ax.plot(strikes, open_interests, 'r-', label='Open interest', 
                color=COLOR_LIST[column_index], marker='.', linewidth=0, markersize=7)

        open_interest_plot_limits[option_type] = ax.get_ylim()
        strike_plot_limits.append(ax.get_xlim())
        ax.set_xlabel('Strike', fontsize=MY_FONTSIZE, labelpad=LABEL_PAD)
        if column_index == 0:
            ax.set_ylabel('Open interest', fontsize=MY_FONTSIZE, labelpad=LABEL_PAD)
        else:
            ax.tick_params(axis='y', which='both', left=True, right=False, labelleft=False)

    # --- in both rows: setting plot limits equal ---
    implied_vol_plot_min_new = 0.0
    implied_vol_plot_max_new = max(implied_vol_plot_max.values())
    outstanding_interest_plot_min_new = min([y_limits[0] for y_limits in open_interest_plot_limits.values()])
    outstanding_interest_plot_max_new = max([y_limits[1] for y_limits in open_interest_plot_limits.values()])
    strike_plot_min_new = max([limits[0] for limits in strike_plot_limits])
    strike_plot_max_new = max([limits[1] for limits in strike_plot_limits])
    
    for column_index, option_type in enumerate(OPTION_TYPES):
        ax = axes[0][column_index]
        ax.set_xlim((strike_plot_min_new, strike_plot_max_new))
        ax.set_ylim((implied_vol_plot_min_new, implied_vol_plot_max_new))
        ax.text(0.96 * spot, 0.6 * implied_vol_plot_max_new, 
                'Spot price', rotation='vertical', fontsize=MY_FONTSIZE)
        ax = axes[1][column_index]
        ax.set_xlim((strike_plot_min_new, strike_plot_max_new))
        ax.set_ylim((outstanding_interest_plot_min_new, outstanding_interest_plot_max_new))
    
    plt.subplots_adjust(wspace=SUBPLOTS_WIDTH_SPACE, hspace=SUBPLOTS_HEIGHT_SPACE)
    plt.show()

    
options_data = read_options_data(par=PAR)        
plot_options_implied_volatility(par=PAR, options_data=options_data)    

{'call': [{'strike': 70.0, 'open interest': 79, 'implied volatility': 4.2422}, {'strike': 75.0, 'open interest': 154, 'implied volatility': 4.0469}, {'strike': 80.0, 'open interest': 107, 'implied volatility': 3.7891}, {'strike': 85.0, 'open interest': 71, 'implied volatility': 3.625}, {'strike': 90.0, 'open interest': 191, 'implied volatility': 3.4727}, {'strike': 95.0, 'open interest': 91, 'implied volatility': 3.3281}, {'strike': 100.0, 'open interest': 719, 'implied volatility': 3.2539}, {'strike': 105.0, 'open interest': 24, 'implied volatility': 3.0547}, {'strike': 110.0, 'open interest': 136, 'implied volatility': 4.3184}, {'strike': 115.0, 'open interest': 33, 'implied volatility': 2.8125}, {'strike': 120.0, 'open interest': 108, 'implied volatility': 2.6953}, {'strike': 125.0, 'open interest': 129, 'implied volatility': 2.5859}, {'strike': 130.0, 'open interest': 128, 'implied volatility': 2.582}, {'strike': 135.0, 'open interest': 36, 'implied volatility': 2.3828}, {'strike':

NameError: name 'FIGURE_SIZE' is not defined

## Analyze the result

* Asymmetric smile and open interest

* Differences between Calls and Puts
